In [1]:
import pandas as pd
import xlrd
import pickle #modulo usado para serializar dados para o arquivo binario
import os
import time

df = pd.read_excel('Hot Stuff.xlsx')

In [11]:
#esta celula calcula o quao popular foi uma musica no mes especificado (Maio de 1990) atribuindo um sistema de pontos baseado em que posicao ela ficou no top 100 em cada semana do mês
#vai contar o quao popular foi a musica no ano atribuindo pontos a quantas semanas ela ficou entre as mais tocadas
#testando se o char especificado esta na entrada atual
#pode ser usado para identificar anos diferentes

tempo = time.time() #Armazena o tempo de inicio do processamento
for ano in range(1960,2015):
    contador = {}
    #Salva os registros do ano atual
    for i in range(len(df)):
        if (str(ano) + '-') in df['WeekID'][i]: #filtra todas as entradas por ano E mes nesse caso (Maio de 1990)
            if(df['SongID'][i] in contador): #testa se a musica ja esta no dicionario (ja foi computada)
                contador[df['SongID'][i]]['Pontos'] += (101 - df['Week Position'][i])
            else: #cai aqui se ainda nao foi computada
                contador[df['SongID'][i]] = {}
                contador[df['SongID'][i]]['Artista'] = df['Performer'][i]
                contador[df['SongID'][i]]['Titulo'] = df['Song'][i]
                contador[df['SongID'][i]]['Ano'] = ano
                contador[df['SongID'][i]]['Pontos'] = (101 - df['Week Position'][i])
                #A medida 'Peak' nao eh a posicao exata que ela atingiu mas os "pontos maximos" ganhos em uma semana (de 1 a 100)
                contador[df['SongID'][i]]['Peak'] = (101 - df['Peak Position'][i]) #salva qual foi a maior posicao alcançada pela musica (usada para criterios de desempate)
                contador[df['SongID'][i]]['Semanas'] = df['Weeks on Chart'][i] #Terceiro criterio de desempate
    
    #Faz o sort do ano atual
    items = sorted(contador.items(), key = lambda tup: (tup[1]["Pontos"], tup[1]["Peak"], tup[1]["Semanas"]), reverse=True)
    
    #armazena as primeiras 100 entradas em um dicionário mais organizado
    top = [dict() for x in range(200)] #cria uma lista de 100 dicionarios
    for i in range(len(top)):
        for y in items[i][1]:
            top[i][y] = items[i][1][y]
            
    #Concatena o dicionario no arquivo teste.bin
    filename = 'teste.bin'
    topBytes = []

    if os.path.exists(filename): #confere se o arquivo existe
        with open(filename,'rb') as openfile:  #with automaticamente da um close() no final
            topBytes = pickle.load(openfile)


    newData = top #Aqui vem a nova data que deve ser concatenada
        
    topBytes = topBytes + newData #concatena o que ja tinha no arquivo binario com a nova data
    with (open(filename, 'wb+')) as openfile: #abre o arquivo de teste no modo de leitura binaria 'wb' como openfile
        pickle.dump(topBytes, openfile) 
        
        
    #Cria o arquivo de indice para consultas    
    filename = 'indices.bin'
    fileBytes = []

    if os.path.exists(filename): #confere se o arquivo existe
        with open(filename,'rb') as openfile:  #with automaticamente da um close() no final
            fileBytes = pickle.load(openfile)


    newData = {'Ano': ano, 'Min': len(topBytes) - 200, 'Max': len(topBytes) - 1} #Aqui vem a nova data que deve ser concatenada
        
    fileBytes.append(newData)#concatena o que ja tinha no arquivo binario com a nova data
    with (open(filename, 'wb+')) as openfile: #abre o arquivo de teste no modo de leitura binaria 'wb' como openfile
        pickle.dump(fileBytes, openfile)
        
t = time.time() - tempo #Salva em t o tempo que o processo levou
print('Tempo levado para gerar os arquivos: ', t, 'segundos')

Tempo levado para gerar os arquivos:  42.311931848526 segundos


In [12]:
#Faz a leitura do arquivo de registros
with (open('teste.bin', 'rb')) as openfile:
    leitura = pickle.load(openfile)
    
with (open('indices.bin', 'rb')) as openfile:
    indices = pickle.load(openfile)

In [13]:
for i in indices:
    print(i)

{'Ano': 2010, 'Min': 0, 'Max': 199}
{'Ano': 2011, 'Min': 200, 'Max': 399}
{'Ano': 2012, 'Min': 400, 'Max': 599}
{'Ano': 2013, 'Min': 600, 'Max': 799}
{'Ano': 2014, 'Min': 800, 'Max': 999}


In [14]:
print(len(leitura))

1000


### Teste da relação
consultando o indice inicial de um dado ano no arquivo indice e imprimindo os 5 mais populares daquele ano

In [15]:
start = 0

for x in indices:
    if x['Ano'] == 2014:
        start = x['Min']
 
for i in range(5):
    print(leitura[start + i])

{'Artista': 'John Legend', 'Titulo': 'All Of Me', 'Ano': 2014, 'Pontos': 4069, 'Peak': 100, 'Semanas': 59}
{'Artista': 'Pharrell Williams', 'Titulo': 'Happy', 'Ano': 2014, 'Pontos': 3898, 'Peak': 100, 'Semanas': 47}
{'Artista': 'Katy Perry Featuring Juicy J', 'Titulo': 'Dark Horse', 'Ano': 2014, 'Pontos': 3816, 'Peak': 100, 'Semanas': 57}
{'Artista': 'OneRepublic', 'Titulo': 'Counting Stars', 'Ano': 2014, 'Pontos': 3373, 'Peak': 99, 'Semanas': 68}
{'Artista': 'Iggy Azalea Featuring Charli XCX', 'Titulo': 'Fancy', 'Ano': 2014, 'Pontos': 2993, 'Peak': 100, 'Semanas': 39}


In [16]:
start = 0

for x in indices:
    if x['Ano'] == 2014:
        start = x['Min']
        
#Retorna as posicoes de 100 a 104 
desvio = 100

for i in range(5):
    print(leitura[start + i + desvio])

{'Artista': 'Ty Dolla $ign Featuring B.o.B', 'Titulo': 'Paranoid', 'Ano': 2014, 'Pontos': 933, 'Peak': 72, 'Semanas': 20}
{'Artista': 'Fall Out Boy', 'Titulo': 'Centuries', 'Ano': 2014, 'Pontos': 924, 'Peak': 91, 'Semanas': 34}
{'Artista': 'Miley Cyrus', 'Titulo': 'Adore You', 'Ano': 2014, 'Pontos': 920, 'Peak': 80, 'Semanas': 18}
{'Artista': 'Nicki Minaj', 'Titulo': 'Pills N Potions', 'Ano': 2014, 'Pontos': 916, 'Peak': 77, 'Semanas': 18}
{'Artista': 'Miranda Lambert Duet With Carrie Underwood', 'Titulo': "Somethin' Bad", 'Ano': 2014, 'Pontos': 898, 'Peak': 82, 'Semanas': 20}
